# *openQmin* script generator

This generates *runHelper.py* automatically from the command-line arguments defined in *openQmin.cpp*.

In [5]:
with open('../openQmin.cpp','r') as f:
    openQmincpp = f.readlines()
arglines = [ line for line in openQmincpp if 'ValueArg' in line ]    

pythonlines = ["""
\"\"\"
This Python script simply calls openQmin.out in command-line mode, 
with all available command-line arguments presented as a Python dictionary.
Variables with empty-string values are ignored, as are commented-out lines. 
\"\"\"

import os
from initHelper import partition_processors

mpi_num_processes = 1

params = dict("""]

for line in arglines[1:]:
    shortname, longname, hint, _, default, vartype = [ eval(item)
        for item in ''.join(
            ''.join(
                line.split('(')[1:]
            ).split(')')[:-1]
        ).replace('false','False').replace('true', 'True').split(',')[:-1]
    ]
    if vartype == 'scalar':
        vartype = 'float'
    elif vartype == 'string':
        default = '\"\"'
    pythonline = f'\t{longname} = {default}, # <{vartype}> {hint}'
    pythonlines.append(pythonline)
pythonlines.append('')
pythonlines.append('\t### The following override Lx, Ly, Lz if and only if do_partition==True in get_runcmd')
pythonlines.append('\twhole_Lx = 50, # <int> number of lattice sites in x direction *before* MPI subdivision')
pythonlines.append('\twhole_Ly = 50, # <int> number of lattice sites in y direction *before* MPI subdivision')
pythonlines.append('\twhole_Lz = 50, # <int> number of lattice sites in z direction *before* MPI subdivision')
pythonlines.append(')')
pythonscript = '\n'.join(pythonlines)

pythonscript += """

  
def get_runcmd(do_partition=True):
    if mpi_num_processes > 1:
        runcmd = f'mpirun -n {mpi_num_processes} '
        if do_partition: 
            # calculate partitioned simulation box size automatically 
            # from entire simulation box size and number of MPI processes
            Rx, Ry, Rz = partition_processors(mpi_num_processes)
            params['Lx'] = int(params['whole_Lx'] // Rx)
            params['Ly'] = int(params['whole_Ly'] // Ry)
            params['Lz'] = int(params['whole_Lz'] // Rz)            
    else:
        runcmd = ''
    runcmd += '../build/openQmin.out ' + ' '.join([
        f'--{key} {val}' for key, val in zip(
            params.keys(), params.values()
        )
        if len(str(val)) > 0
        and not key in [
            'whole_Lx', 'whole_Ly', 'whole_Lz'
        ]
    ])
    return runcmd
"""

with open ('runHelper.py', 'w') as f:
    f.write(pythonscript)